In [4]:
from flask import Flask, render_template, request
from keras.models import load_model
import tensorflow as tf
#from tensorflow.keras.utils import load_img
from keras.utils import load_img , img_to_array
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pymongo
from flask_pymongo import pymongo
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import base64
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import io
import imghdr
import cv2
from io import BytesIO


app = Flask(__name__,template_folder='templates',static_url_path='/static')

class_indices ={0 : 'AlgaeBloom', 1 : 'MarineDebris', 2 : 'OilSpill'}


model = load_model('model.h5')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.make_predict_function()


#Connect to MongoDB database
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['finaldb']
    
# Load all feature vectors from the database into a NumPy array
def get_database_features(class_label):
    #Connect to MongoDB database
    client = pymongo.MongoClient('mongodb://localhost:27017/')
    db = client['finaldb']
    
    
    #get database features array
    features_list = []
    images_collection = db[class_label]
    for image in images_collection.find():
        features_list.append(image['features'])
    features_array = np.array(features_list)
    features_array=features_array.reshape((features_array.shape[0],-1))
    return features_array


def predict_input(img_path):
    img = load_img(img_path, target_size=(224, 224))
#     num_channels=len(img.getbands())
#     if num_channels !=3:
#         img=img.convert("RGB")
    img = img.convert("L")
    img = Image.merge("RGB", (img, img, img))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array/255
    input_features = model.predict(img_array)
    input_features=input_features.reshape((input_features.shape[0],-1))
    
   # print(input_features)
    return input_features
    
# Get the class label
def get_class_label(input_features):
    class_idx = np.argmax(input_features[0])
    print(class_idx)
    print(class_indices)
    class_label = list(class_indices.values())[class_idx]
    print("The image belongs to the class:", class_label)
    return class_label
    
def get_similar_images(input_features, features_array):
    input_features = input_features.reshape(1, -1)
    similarities = []
    sim_score=[]
    for features in features_array:
        similarity = cosine_similarity(input_features, features.reshape(1, -1))[0, 0]
        similarities.append(similarity)
    # Find the indices of the top 5 most similar images
    top_indices = np.argsort(similarities)[-5:][::-1]
    # Print the similarity scores for the top 5 most similar images
    for i in top_indices:
        sim_score.append(similarities[i])
    
    print("print similarity score",sim_score)
    return top_indices,sim_score



# routes
@app.route("/", methods=['GET', 'POST'])
def index():
    images=[]
    
    return render_template("output.html",images=images)


@app.route("/submit", methods = ['GET', 'POST'])
def get_output():
    input_features = None
    similarities = None # Initialize similarities to None
    if request.method == 'POST':
        img = request.files['my_image']

        img_path = "static/" + img.filename	
        print(img_path)
        img.save(img_path)  
        

        input_features = predict_input(img_path)
       
        class_label = get_class_label(input_features)
        
        features_array = get_database_features(class_label)
        
        
        get_similar_images(input_features, features_array)
        top_indices , sim_score= get_similar_images(input_features, features_array)
        similar_indices = top_indices.tolist()
    
        class_label = get_class_label(input_features)
        image_collection = db[class_label]
        print(image_collection)

        print(top_indices)
        images = []
        metadata = []
        data_list=[] 
        base64_strings=[]

        for index in top_indices:
#             print(index)
            doc =image_collection.find_one({'index': int(index)})
            print("hbaj",doc)

            if "base64_string" in doc:
                base64_string = doc["base64_string"]
                if base64_string:
                    print(f"Base64 string for index {index} calculated successfully")
                    base64_strings.append(base64_string)
              
            if(class_label=='MarineDebris'):
                meta= image_collection.find_one({'index': int(index)},{"Date":1,"latlong":1})
                if meta:
                    metadata.append({

                        'Date': meta['Date'],
                        'latlong':meta['latlong'],


                        })
            else:
                meta= image_collection.find_one({'index': int(index)},{"location":1,"Date":1,"latlong":1,"Satellite":1})
                if meta:
                    metadata.append({
                        'location':meta['location'],
                        'Date': meta['Date'],
                        'latlong':meta['latlong'],
                        'Satellite':meta['Satellite']

                        })
                    
        print("metedata printed")            
        print("get similar images")
        
        print(metadata)
        
        
    return render_template("output.html",sim_score=sim_score,meta=meta,base64_string=base64_string,base64_strings=base64_strings, index=index,images=images, metadata=metadata,class_label=class_label,img_path=img_path,top_indices=top_indices)

    
if __name__ =="__main__":
	#app.debug = True
    app.run()

ModuleNotFoundError: No module named 'flask_pymongo'

In [2]:
pip install pymongo



     -------------------------------------- 472.9/472.9 kB 1.2 MB/s eta 0:00:00
     ------------------------------------ 307.7/307.7 kB 576.5 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\DELL\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [5]:
pip install flask_pymango


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement flask_pymango (from versions: none)
ERROR: No matching distribution found for flask_pymango


In [3]:
-m pip install --upgrade pip


SyntaxError: invalid syntax (294659197.py, line 1)

In [17]:
# import os
# from PIL import Image

# directory = r"C:\Users\DELL\Desktop\yjftvjhb"
# for filename in os.listdir(directory):
#     if filename.endswith(".jpg") or filename.endswith(".png"):
#         image = Image.open(os.path.join(directory, filename))
#         image = image.convert("RGB")  # Remove the alpha channel
#         new_filename = os.path.splitext(filename)[0] + ".jpeg"
#         image.save(os.path.join(directory, new_filename), "JPEG")